# 小项目：飞船打陨石游戏

## 绘制静态图像

In [12]:
from random import *
from math import *
from part1lib import *
import pygame, sys


class PolygonModel:
    def __init__(self, points):
        self.points = points
        self.rotation_angle = 0
        self.x = 0
        self.y = 0

    def transformed(self):
        return tuple(rotate(self.rotation_angle, (add(p, (self.x, self.y)) for p in self.points)))

class Ship(PolygonModel):
    def __init__(self):
        super().__init__([(0.5, 0), (-0.25, 0), (-0.25, -0.25)])

class Asteroid(PolygonModel):
    def __init__(self):
        sides = randint(5, 9)
        vs = [to_cartesian((uniform(0.5, 1.0), 2*pi*i/sides)) for i in range(sides)]
        super().__init__(vs)


def to_pixels(x, y):
    return ((x+10)*20, (-y+10)*20)

GREEN = (0, 255, 0)
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
def draw_poly(screen, polygon_model, color=GREEN):
    pixels_points = [to_pixels(x, y) for x, y in polygon_model.transformed()]
    pygame.draw.aalines(screen, color, True, pixels_points, 10)

# main loop
ship = Ship()

asteroid_count = 10
asteroids = [Asteroid() for _ in range(asteroid_count)]
for ast in asteroids:
    ast.x = randint(-9, 9)
    ast.y = randint(-9, 9)

pygame.init()

size = width, height = 400, 400
screen = pygame.display.set_mode(size)
pygame.display.set_caption("Asteroid!")

done = False
clock = pygame.time.Clock()
while not done:
    clock.tick(60) # 设定每秒帧率，节省CPU资源
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            done = True
    
    screen.fill(BLACK) # set BG Color

    draw_poly(screen, ship)
    for ast in asteroids:
        draw_poly(screen, ast)

    pygame.display.flip()

pygame.quit()

## 让物体运动起来

In [6]:
from random import *
from math import *
from part1lib import *
import pygame, sys


class PolygonModel:
    def __init__(self, points):
        self.points = points
        self.rotation_angle = 0
        self.x = 0
        self.y = 0
        # 添加速度参数
        self.vx = 0
        self.vy = 0

    def rotate(self, angle):
        angle %= 2 * pi
        self.rotation_angle = (self.rotation_angle + 2 * pi + angle) % (2 * pi)

    def move(self, milliseconds):
        dx, dy = (
            self.vx * milliseconds / 1000.0,
            self.vy * milliseconds / 1000.0,
        )
        self.x, self.y = add((self.x, self.y), (dx, dy))

        # 防止飞出屏幕区域
        if self.x < -10 or self.x > 10:
            self.vx = -self.vx 
        if self.y < -10 or self.y > 10:
            self.vy = -self.vy

    def transformed(self):
        return tuple(add(p, (self.x, self.y)) for p in rotate(self.rotation_angle, self.points))

class Ship(PolygonModel):
    def __init__(self):
        super().__init__([(0.5, 0), (-0.25, 0.25), (-0.25, -0.25)])

class Asteroid(PolygonModel):
    def __init__(self):
        sides = randint(5, 9)
        vs = [to_cartesian((uniform(0.5, 1.0), 2*pi*i/sides)) for i in range(sides)]
        super().__init__(vs)
        # 让小行星无规律的运动，速度方向为-1，1之间
        self.vx = uniform(-1, 1)
        self.vy = uniform(-1, 1)


def to_pixels(x, y):
    return ((x+10)*20, (-y+10)*20)

RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
def draw_poly(screen, polygon_model, color=GREEN):
    pixels_points = [to_pixels(x, y) for x, y in polygon_model.transformed()]
    pygame.draw.aalines(screen, color, True, pixels_points, 10)

# main loop
ship = Ship()
ACCELERATION = 3
ROTATE_ANGLE = 2 * pi * 15 / 360 # 15°

asteroid_count = 10
asteroids = [Asteroid() for _ in range(asteroid_count)]
for ast in asteroids:
    ast.x = randint(-9, 9)
    ast.y = randint(-9, 9)

pygame.init()

size = width, height = 400, 400
screen = pygame.display.set_mode(size)
pygame.display.set_caption("Asteroid!")

done = False
clock = pygame.time.Clock()
while not done:
    clock.tick(60) # 设定每秒帧率，节省CPU资源
    milliseconds = clock.get_time() # 计算从上一帧到现在过去了多长时间

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            done = True
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_UP:
                acceleration = ACCELERATION
            if event.key == pygame.K_DOWN:
                acceleration = -ACCELERATION
            if event.key == pygame.K_LEFT:
                ship.rotate(ROTATE_ANGLE)
                acceleration = 0
            if event.key == pygame.K_RIGHT:
                ship.rotate(-ROTATE_ANGLE)
                acceleration = 0

            ax = acceleration * cos(ship.rotation_angle)
            ay = acceleration * sin(ship.rotation_angle)
            ship.vx += ax * milliseconds / 1000.0
            ship.vy += ay * milliseconds / 1000.0
    
    screen.fill(BLACK) # set BG Color

    ship.move(milliseconds)
    draw_poly(screen, ship, color=RED)
    for ast in asteroids:
        ast.move(milliseconds)
        draw_poly(screen, ast)

    pygame.display.flip()

pygame.quit()

## 模拟力场添加黑洞 